In [1]:
import pandas as pd 
import numpy as np

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer, SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import os

In [2]:
from sklearn.base import TransformerMixin, BaseEstimator

In [3]:
from sklearn.impute import SimpleImputer
import pickle

In [4]:
def read_data(file):
    """
    
    """
    df = (pd.read_csv(file, dtype = {
                            "PassengerId":np.int32,
                            "Name": "object",
                            "Pclass":np.int,
                            "Survived":np.int32,
                            "Sex":"object",
                            "Age":np.float,
                            "SibSp":np.int,
                            "Embarked":"object",
                            "Cabin":"object",
                            "Fare":np.float64,
                        },)
            .rename(columns=str.lower)
            .set_index("passengerid")
            .rename(columns=str.lower)
            .drop(['cabin','name','ticket'],axis=1)
        )
    return df
    

In [6]:
training_data = read_data("../data/train.csv")

In [7]:
categorical_transformer = Pipeline([ 
                                    ('onehot', OneHotEncoder(handle_unknown='error', drop="first"))])

numeric_transformer = Pipeline([('impute', SimpleImputer()),
                                ('sc', StandardScaler())])

In [8]:
preprocessor = ColumnTransformer(
    transformers=[ 
        ('cat', categorical_transformer, ['sex']),
        ('num', numeric_transformer, ['age'])
        ])

In [9]:
rf_pipe = Pipeline([('pre', preprocessor),
                    ('rf', RandomForestClassifier(n_estimators=300))])

In [10]:
rf_pipe.fit(training_data.drop(['survived'],axis=1),training_data.survived)

Pipeline(memory=None,
         steps=[('pre',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('onehot',
                                                                   OneHotEncoder(categories='auto',
                                                                                 drop='first',
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='error',
                                                                                 sparse=True))],
                                                       

In [17]:
os.listdir("../models/")

[]

In [11]:
os.listdir('../../titanic-prod/models')

['rf_pipe.pkl']

In [23]:
with open('../models/rf_pipe.pkl','wb') as h:
    pickle.dump(rf_pipe,h)

In [25]:
with open('../models/rf_pipe.pkl','rb') as f:
    mm = pickle.load(f)

In [26]:
training_data.drop(['survived'],axis=1).loc[1,:].to_json()

'{"pclass":3,"sex":"male","age":22.0,"sibsp":1,"parch":0,"fare":7.25,"embarked":"S"}'

In [27]:
training_data.drop(['survived'],axis=1)

,pclass,sex,age,sibsp,parch,fare,embarked
passengerid,,,,,,,
1,3,male,22.0,1,0,7.2500,S
2,1,female,38.0,1,0,71.2833,C
3,3,female,26.0,0,0,7.9250,S
4,1,female,35.0,1,0,53.1000,S
5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
887,2,male,27.0,0,0,13.0000,S
888,1,female,19.0,0,0,30.0000,S
889,3,female,NaN,1,2,23.4500,S


In [28]:
str(mm.predict(training_data.drop(['survived'],axis=1).sample(1))[0])

'0'